# Zero-Shot Prompting Tutorial

## Overview

This tutorial provides a comprehensive introduction to zero-shot prompting, a powerful technique in prompt engineering that allows language models to perform tasks without specific examples or prior training. We'll explore how to design effective zero-shot prompts and implement strategies using OpenAI's GPT models and the LangChain library.

## Motivation

Zero-shot prompting is crucial in modern AI applications as it enables language models to generalize to new tasks without the need for task-specific training data or fine-tuning. This capability significantly enhances the flexibility and applicability of AI systems, allowing them to adapt to a wide range of scenarios and user needs with minimal setup.

## Key Components

1. **Understanding Zero-Shot Learning**: An introduction to the concept and its importance in AI.
2. **Prompt Design Principles**: Techniques for crafting effective zero-shot prompts.
3. **Task Framing**: Methods to frame various tasks for zero-shot performance.
4. **OpenAI Integration**: Using OpenAI's GPT models for zero-shot tasks.
5. **LangChain Implementation**: Leveraging LangChain for structured zero-shot prompting.

## Method Details

The tutorial will cover several methods for implementing zero-shot prompting:

1. **Direct Task Specification**: Crafting prompts that clearly define the task without examples.
2. **Role-Based Prompting**: Assigning specific roles to the AI to guide its responses.
3. **Format Specification**: Providing output format guidelines in the prompt.
4. **Multi-step Reasoning**: Breaking down complex tasks into simpler zero-shot steps.
5. **Comparative Analysis**: Evaluating different zero-shot prompt structures for the same task.

Throughout the tutorial, we'll use Python code with OpenAI and LangChain to demonstrate these techniques practically.

## Conclusion

By the end of this tutorial, learners will have gained:

1. A solid understanding of zero-shot prompting and its applications.
2. Practical skills in designing effective zero-shot prompts for various tasks.
3. Experience in implementing zero-shot techniques using OpenAI and LangChain.
4. Insights into the strengths and limitations of zero-shot approaches.
5. A foundation for further exploration and innovation in prompt engineering.

This knowledge will empower learners to leverage AI models more effectively across a wide range of applications, enhancing their ability to solve novel problems and create more flexible AI systems.

## Setup

Let's start by importing the necessary libraries and setting up our environment.

In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set up OpenAI API key
# os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
# Initialize the language model
# llm = ChatOpenAI(model="gpt-4o-mini")

from select_llm import get_llm, get_supported_ollama_models
ollama_model = "mistral:latest"  # or None for OpenAI
# ollama_model = "llama3.2:1b"  # or None for OpenAI
# ollama_model = "tinyllama:latest"  # or None for OpenAI
# ollama_model = "deepseek-r1:7b"
llm = get_llm(ollama_model) #,force_port=12434)
if llm is None:
    models = get_supported_ollama_models()
    print("Available Ollama models:", models)

def create_chain(prompt_template, print_prompt=False):
    """
    Create a LangChain chain with the given prompt template.
    
    Args:
        prompt_template (str): The prompt template string.
        print_prompt (bool): If True, print the formatted prompt when invoke() is called.
    
    Returns:
        LangChain LCEL chain (prompt | llm).
    """
    prompt = PromptTemplate.from_template(prompt_template)
    chain = prompt | llm
    if print_prompt:
        class ChainWithPrintPrompt:
            def invoke(self, inputs, *args, **kwargs):
                print("--- Prompt sent to model ---")
                print(prompt.format(**inputs))
                print("---")
                return chain.invoke(inputs, *args, **kwargs)
        return ChainWithPrintPrompt()
    return chain

Using Ollama on MAC (port 12434).


## 1. Direct Task Specification

In this section, we'll explore how to craft prompts that clearly define the task without providing examples. This is the essence of zero-shot prompting.

In [ ]:
direct_task_prompt = """Classify the sentiment of the following text as positive, negative, or neutral.
Do not explain your reasoning, just provide the classification.

Text: {text}

Sentiment:"""

direct_task_chain = create_chain(direct_task_prompt)

# Test the direct task specification
texts = [
    "I absolutely loved the movie! The acting was superb.",
    "The weather today is quite typical for this time of year.",
    "I'm disappointed with the service I received at the restaurant."
]

for text in texts:
    result = direct_task_chain.invoke({"text": text}).content
    print(f"Text: {text}")
    print(f"Sentiment: {result}")

## 2. Format Specification

Providing output format guidelines in the prompt can help structure the AI's response in a zero-shot scenario.

In [2]:
format_spec_prompt = """Generate a short news article about {topic}. 
Structure your response in the following format:

Headline: [A catchy headline for the article]

Lead: [A brief introductory paragraph summarizing the key points]

Body: [2-3 short paragraphs providing more details]

Conclusion: [A concluding sentence or call to action]

IMPORTANT: {important_note}
"""

format_spec_chain = create_chain(format_spec_prompt)

# Test the format specification prompting
topic = "The discovery of a new earth-like exoplanet"
note = "Always use these exact section labels on their own lines: Headline:, Lead:, Body:, Conclusion:. Include 'Body:' immediately before the 2-3 detail paragraphs."
result = format_spec_chain.invoke({"topic": topic, "important_note": note}).content
print(result)

 Headline: A New Earth-Like Exoplanet Discovered in Habitable Zone - Kepler-452b Could be Our Cosmic Twin!

Lead: Astronomers have announced the discovery of a new exoplanet, Kepler-452b, located within the habitable zone of its star and displaying characteristics remarkably similar to Earth. This exciting find offers tantalizing possibilities for future study and could potentially harbor life beyond our own planet.

Body:

1. Location and Orbit: Kepler-452b resides in a solar system approximately 1,400 light years from Earth, orbiting its star within the habitable zone - the area where liquid water may exist on the planet's surface. This position suggests that conditions are suitable for life as we know it.

2. Size and Temperature: The exoplanet has a radius that is 60% larger than Earth, leading scientists to believe it may have a dense atmosphere similar to that of Venus. However, recent data indicates that its surface temperature could be relatively mild, with an average of 20 deg

## 3. Multi-step Reasoning

For complex tasks, we can break them down into simpler zero-shot steps. This approach can improve the overall performance of the model.

In [ ]:
multi_step_prompt = """Analyze the following text for its main argument, supporting evidence, and potential counterarguments. 
Provide your analysis in the following steps:

1. Main Argument: Identify and state the primary claim or thesis.
2. Supporting Evidence: List the key points or evidence used to support the main argument.
3. Potential Counterarguments: Suggest possible objections or alternative viewpoints to the main argument.

Text: {text}

Analysis:"""

multi_step_chain = create_chain(multi_step_prompt,print_prompt=True)

# Test the multi-step reasoning approach
text = """While electric vehicles are often touted as a solution to climate change, their environmental impact is not as straightforward as it seems. 
The production of batteries for electric cars requires significant mining operations, which can lead to habitat destruction and water pollution. 
Moreover, if the electricity used to charge these vehicles comes from fossil fuel sources, the overall carbon footprint may not be significantly reduced. 
However, as renewable energy sources become more prevalent and battery technology improves, electric vehicles could indeed play a crucial role in combating climate change."""

result = multi_step_chain.invoke({"text": text}).content
print(result)

## 4. Comparative Analysis

Let's compare different zero-shot prompt structures for the same task to evaluate their effectiveness.

In [ ]:
def compare_prompts(task, prompt_templates, print_prompt=False):
    """
    Compare different prompt templates for the same task.
    
    Args:
        task (str): The task description or input.
        prompt_templates (dict): A dictionary of prompt templates with their names as keys.
        print_prompt (bool): If True, print the formatted prompt for each template before the result.
    """
    print(f"Task: {task}\n")
    for name, template in prompt_templates.items():
        chain = create_chain(template, print_prompt=print_prompt)
        result = chain.invoke({"task": task}).content
        print(f"{name} Prompt Result:")
        print(result)
        print("\n" + "-"*50 + "\n")

task = "Explain concisely the concept of blockchain technology"

prompt_templates = {
    "Basic": "Explain {task}.",
    "Structured": """Explain {task} by addressing the following points:
1. Definition
2. Key features
3. Real-world applications
4. Potential impact on industries"""
}

compare_prompts(task, prompt_templates,print_prompt=True)